In [47]:
# system imports
import glob
import os

# feature extractoring and preprocessing data
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
pd.plotting.register_matplotlib_converters()
from PIL import Image
import pathlib
import csv
from tqdm.auto import tqdm
from sklearn.utils import resample
import seaborn as sns

#Keras
import keras
import warnings
warnings.filterwarnings('ignore')

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, scale, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.model_selection import KFold
import librosa, librosa.display, os, csv
import matplotlib
import pylab
plt.switch_backend('agg')
import itertools
import scipy as sp
from scipy import signal
import joblib
from glob import glob
import urllib

# internal imports
from utils import preproces, CoughNet

In [48]:
# Kaggle Dataset - Exploration
fn_dataset = 'data/cough_trial_extended.csv'
df_dataset = pd.read_csv(fn_dataset)

print('Total number of examples:', len(df_dataset))
print('Number of positive examples:', len(df_dataset[df_dataset['class'] == 'covid']))
print('Number of negative examples:', len(df_dataset[df_dataset['class'] == 'not_covid']))

df_dataset

Total number of examples: 170
Number of positive examples: 19
Number of negative examples: 151


,file_properties,class
0,0v8MGxNetjg_ 10.000_ 20.000.wav,not_covid
1,1j1duoxdxBg_ 70.000_ 80.000.wav,not_covid
2,1MSYO4wgiag_ 120.000_ 130.000.wav,not_covid
3,1PajbAKd8Kg_ 0.000_ 10.000.wav,not_covid
4,cov1.wav,covid
...,...,...
165,-bZrDCS8KAg_ 70.000_ 80.000.wav,not_covid
166,-ej81N6Aqo4_ 0.000_ 8.000.wav,not_covid
167,-gvLnl1smfs_ 90.000_ 100.000.wav,not_covid
168,-hu5q-Nn4BM_ 70.000_ 80.000.wav,not_covid


In [49]:
# Kaggle Dataset - Feature Extraction
df_features_cols = ['filename', 'chroma_stft', 'rmse', 'spectral_centroid', 'spectral_bandwidth', 'rolloff', 'zero_crossing_rate']
for i in range(1, 21):
    df_features_cols.append(f'mfcc{i}')
df_features_cols.append('label')

df_features = pd.DataFrame(columns=df_features_cols)

for row_index, row in tqdm(df_dataset.iterrows(), total=len(df_dataset)):
    fn_wav = os.path.join('data/trial_covid/', row['file_properties'])
    feature_row = preproces(fn_wav)
    feature_row['filename'] = row['file_properties']
    feature_row['label'] = row['class']
    df_features = df_features.append(feature_row, ignore_index=True)

df_features.to_csv('data/prepared_data_kaggle.csv', index=False, columns=df_features_cols)

df_features.head()

  0%|          | 0/170 [00:00<?, ?it/s]

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,0v8MGxNetjg_ 10.000_ 20.000.wav,0.519951,0.045853,1612.895795,1411.838677,2907.580566,0.107019,-376.876007,111.017372,-31.904015,...,-7.439712,-1.034580,-0.203083,-3.513495,-1.745705,-3.011878,-2.878482,-2.106427,-4.026825,not_covid
1,1j1duoxdxBg_ 70.000_ 80.000.wav,0.535472,0.001771,2892.087076,2467.408141,5072.664388,0.148584,-519.158447,60.781284,-13.722886,...,-0.909973,7.216461,-1.719629,3.903021,3.653039,3.043882,2.439957,2.781968,2.195162,not_covid
2,1MSYO4wgiag_ 120.000_ 130.000.wav,0.496666,0.033657,3429.061935,2788.634413,6886.288452,0.225315,-282.297913,48.581680,-15.522366,...,-6.066336,-4.167640,1.017302,-0.523806,0.538693,-8.855953,-2.927977,-1.118562,-5.906228,not_covid
3,1PajbAKd8Kg_ 0.000_ 10.000.wav,0.407549,0.013452,2710.811637,2664.287550,5778.474935,0.142076,-346.857300,75.765617,-7.648194,...,5.053118,-0.291308,0.987186,-2.447526,3.692367,2.312328,-2.059656,-4.772599,-0.503851,not_covid
4,cov1.wav,0.412697,0.059004,1555.648634,1418.599932,2870.737092,0.133998,-340.588013,104.156700,-32.228443,...,-8.247168,0.940006,-5.701087,-6.326630,-1.080040,-1.812609,-2.518986,-3.684266,-3.564146,covid


In [50]:
#Virufy Dataset - Exploration
import glob
df_dataset = pd.DataFrame(columns=['file_properties', 'class'])
for fn in glob.glob('data/virufy/pos/*.mp3'):
    df_dataset = df_dataset.append({'file_properties': fn, 'class': 'covid'}, ignore_index=True)
for fn in glob.glob('data/virufy/neg/*.mp3'):
    df_dataset = df_dataset.append({'file_properties': fn, 'class': 'not_covid'}, ignore_index=True)

print('Total number of examples:', len(df_dataset))
print('Number of positive examples:', len(df_dataset[df_dataset['class'] == 'covid']))
print('Number of negative examples:', len(df_dataset[df_dataset['class'] == 'not_covid']))

df_dataset

Total number of examples: 121
Number of positive examples: 48
Number of negative examples: 73


,file_properties,class
0,data/virufy/pos\pos-0421-084-cough-m-50-0.mp3,covid
1,data/virufy/pos\pos-0421-084-cough-m-50-1.mp3,covid
2,data/virufy/pos\pos-0421-084-cough-m-50-2.mp3,covid
3,data/virufy/pos\pos-0421-084-cough-m-50-3.mp3,covid
4,data/virufy/pos\pos-0421-084-cough-m-50-4.mp3,covid
...,...,...
116,data/virufy/neg\neg-0422-097-cough-m-37-8.mp3,not_covid
117,data/virufy/neg\neg-0422-097-cough-m-37-9.mp3,not_covid
118,data/virufy/neg\neg-0422-098-cough-f-24-0.mp3,not_covid
119,data/virufy/neg\neg-0422-098-cough-f-24-1.mp3,not_covid


In [5]:
#Virufy Dataset - Feature Extraction
#df_features_cols = ['filename', 'chroma_stft', 'rmse', 'spectral_centroid', 'spectral_bandwidth', 'rolloff', 'zero_crossing_rate']
#for i in range(1, 21):
    #df_features_cols.append(f'mfcc{i}')
#df_features_cols.append('label')

#df_features = pd.DataFrame(columns=df_features_cols)

#for row_index, row in tqdm(df_dataset.iterrows(), total=len(df_dataset)):
    #fn_wav = os.path.join('data/virufy/pos/*.mp3', row['file_properties'])
    #fn_wav = row['file_properties']
    #feature_row = preproces(fn_wav)
    #feature_row['filename'] = row['file_properties']
    #feature_row['label'] = row['class']
    #df_features = df_features.append(feature_row, ignore_index=True)

#df_features.to_csv('data/prepared_data_virufy.csv', index=False, columns=df_features_cols)

#df_features.head()

In [51]:
#Combine Datasets
df_features_kaggle = pd.read_csv('data/prepared_data_kaggle.csv')
df_features_virufy = pd.read_csv('data/prepared_data_virufy.csv')
df_features = pd.concat([df_features_kaggle, df_features_virufy])

df_features.to_csv('data/prepared_data.csv', index=False, columns=df_features_cols)

print('Total number of examples:', len(df_features))
print('Number of positive examples:', len(df_features[df_features['label'] == 'covid']))
print('Number of negative examples:', len(df_features[df_features['label'] == 'not_covid']))

Total number of examples: 291
Number of positive examples: 67
Number of negative examples: 224


In [52]:
#Balanced Dataset
df_features = pd.read_csv('data/prepared_data.csv')

# Separate majority and minority classes
df_majority = df_features[df_features['label'] == 'not_covid']
df_minority = df_features[df_features['label'] == 'covid']
 
# Downsample majority class
df_majority_balanced = resample(df_majority, replace=False, n_samples=len(df_minority), random_state=42)
 
# Combine minority class with downsampled majority class
df_balanced = pd.concat([df_majority_balanced, df_minority])

df_balanced.to_csv('data/prepared_data_balanced.csv', index=False)

print('Total number of examples:', len(df_balanced))
print('Number of positive examples:', len(df_balanced[df_balanced['label'] == 'covid']))
print('Number of negative examples:', len(df_balanced[df_balanced['label'] == 'not_covid']))

Total number of examples: 134
Number of positive examples: 67
Number of negative examples: 67


In [53]:
#Training and Evaluation
# system imports

from datetime import datetime

import torch
from torch.utils.tensorboard import SummaryWriter

# internal imports
from utils import plot_confusion_matrix

# device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [54]:
#Hyperparameters
hparams = {    
    'dataset': 'data/prepared_data_balanced.csv',
    'epochs': 15,
    'batch_size': 16,
    'lr': 1e-3,
    'features': [
        'chroma_stft', 'rmse', 'spectral_centroid', 'spectral_bandwidth', 'rolloff', 'zero_crossing_rate',
        'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9', 'mfcc10', 
        'mfcc11', 'mfcc12', 'mfcc13', 'mfcc14', 'mfcc15', 'mfcc16', 'mfcc17', 'mfcc18', 'mfcc19', 'mfcc20'
    ]
}

In [55]:
#Prepare Data
df_features = pd.read_csv(hparams['dataset'])

X = np.array(df_features[hparams['features']], dtype=np.float32)

encoder = LabelEncoder()
y = encoder.fit_transform(df_features['label'])
print('classes:', encoder.classes_)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print('X_train.shape:', X_train.shape)
print('y_train.shape:', y_train.shape)

# create pytorch dataloader
torch.manual_seed(42)
train_dataset = torch.utils.data.TensorDataset(torch.Tensor(X_train), torch.Tensor(y_train).long())
test_dataset = torch.utils.data.TensorDataset(torch.Tensor(X_test), torch.Tensor(y_test).long())
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=hparams['batch_size'], shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=hparams['batch_size'], shuffle=False)

classes: ['covid' 'not_covid']
X_train.shape: (107, 26)
y_train.shape: (107,)


In [56]:
#Setup Model
# Design model (input, output size, forward pass)
class CoughNet(torch.nn.Module):
    def __init__(self, input_size):
        super(CoughNet, self).__init__()
        self.l1 = torch.nn.Linear(input_size, 512)
        self.l2 = torch.nn.Linear(512, 256)
        self.l3 = torch.nn.Linear(256, 128)
        self.l4 = torch.nn.Linear(128, 64)
        self.l5 = torch.nn.Linear(64, 10)
        self.l6 = torch.nn.Linear(10, 2)

    def forward(self, x):
        x = torch.relu(self.l1(x))
        x = torch.relu(self.l2(x))
        x = torch.relu(self.l3(x))
        x = torch.relu(self.l4(x))
        x = torch.relu(self.l5(x))
        x = self.l6(x)
        return x

model = CoughNet(len(hparams['features'])).to(device)

In [57]:
#Training
# Construct loss and optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=hparams['lr'])
criterion = torch.nn.CrossEntropyLoss()

def train(loader_train, model, optimizer, epoch):
    model.train()
    running_loss = 0.0
    running_correct = 0.0
    total = 0
    pbar = tqdm(enumerate(loader_train), total=len(loader_train))
    for batch_ndx, sample in pbar: 
        features, labels = sample[0].to(device), sample[1].to(device) 

        # forward pass and loss calculation
        outputs = model(features)
        loss = criterion(outputs, labels)  
        
        # backward pass    
        loss.backward()
        
        # update weights
        optimizer.step()
        optimizer.zero_grad()
        
        # calculate metrics
        running_loss += loss.item()
        predictions = torch.argmax(outputs.data, 1)
        running_correct += (predictions == labels).sum().item()

        # print informations
        pbar.set_description(f'[Training Epoch {epoch+1}]') 
        total += labels.shape[0]
        pbar.set_postfix({'loss': running_loss / total, 'train_accuracy': running_correct / total})
        
    # write informations to tensorboard
    writer.add_scalar('Loss/Train', running_loss / total, epoch+1)
    writer.add_scalar('Accuracy/Train', running_correct / total, epoch+1)

def evaluate(loader_test, model, epoch):
    model.eval()
    running_loss = 0.0
    running_correct = 0.0
    total = 0
    with torch.no_grad():
        pbar = tqdm(enumerate(loader_test), total=len(loader_test))
        for batch_ndx, sample in pbar:
            features, labels = sample[0].to(device), sample[1].to(device) 

            # forward pass and loss calculation
            outputs = model(features)
            loss = criterion(outputs, labels)  

            # calculate metrics
            running_loss += loss.item()
            predictions = torch.argmax(outputs.data, 1)
            running_correct += (predictions == labels).sum().item()

            # print informations
            pbar.set_description(f'[Evaluating Epoch {epoch+1}]')
            total += labels.shape[0]
            pbar.set_postfix({'loss': running_loss / total, 'eval_accuracy': running_correct / total})
        
    # write informations to tensorboard
    writer.add_scalar('Loss/Eval', running_loss / total, epoch+1)
    writer.add_scalar('Accuracy/Eval', running_correct / total, epoch+1)

# initialize tensorboard summary writer
time_stamp = datetime.now().strftime('%Y%m%d-%H%M%S')
writer = SummaryWriter(f'logs/{time_stamp}/')

# add graph to tensorboard
features = iter(test_loader).next()[0]
writer.add_graph(model, features)

# training loop
for epoch in range(hparams['epochs']):
    train(train_loader, model, optimizer, epoch)
    evaluate(test_loader, model, epoch)

# close tensorboard
writer.close()

# open tensorboard
# tensorboard --logdir logs

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [58]:
#Plot confusion matrix
# internal imports
from utils import plot_confusion_matrix
model.eval()
with torch.no_grad():
    outputs = model(torch.tensor(X_test))
    predictions = torch.argmax(outputs.data, 1)

plot_confusion_matrix(y_test, predictions, encoder.classes_)
plt.show()

In [59]:
checkpoint = {
    'hparams': hparams,
    'model_state': model.state_dict(),
    'scaler': scaler,
    'encoder': encoder
}
torch.save(checkpoint, 'data/checkpoints/checkpoint.pth')

In [60]:
#Path to test file
fn_wav = 'data/test.wav' # positive example

In [61]:
#Inference
# load model from checkpoint
loaded_checkpoint = torch.load('data/checkpoints/checkpoint.pth')

hparams = loaded_checkpoint['hparams']
scaler = loaded_checkpoint['scaler']
encoder = loaded_checkpoint['encoder']

model = CoughNet(len(hparams['features']))
model.eval()
model.load_state_dict(loaded_checkpoint['model_state'])

# create input features
df_features = pd.DataFrame(columns=hparams['features'])
df_features = df_features.append(preproces(fn_wav), ignore_index=True)
X = np.array(df_features[hparams['features']], dtype=np.float32)
X = torch.Tensor(scaler.transform(X))

outputs = torch.softmax(model(X), 1)
predictions = torch.argmax(outputs.data, 1)

# print result
print(f'model outputs {outputs[0].detach().numpy()} which predicts the class {encoder.classes_[predictions]}!')

model outputs [9.9903202e-01 9.6800784e-04] which predicts the class covid!


In [62]:
#k-Fold Cross Validation
# system imports
import os
from datetime import datetime

# additional imports
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, KFold

# internal imports
from utils import plot_confusion_matrix
# device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [63]:
#Hyperparameters
hparams = {    
    'dataset': 'data/prepared_data_balanced.csv',
    'epochs': 20,
    'batch_size': 16,
    'lr': 1e-3,
    'features': [
        'chroma_stft', 'rmse', 'spectral_centroid', 'spectral_bandwidth', 'rolloff', 'zero_crossing_rate',
        'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9', 'mfcc10', 
        'mfcc11', 'mfcc12', 'mfcc13', 'mfcc14', 'mfcc15', 'mfcc16', 'mfcc17', 'mfcc18', 'mfcc19', 'mfcc20'
    ]
}

In [64]:
#Prepare Data
df_features = pd.read_csv(hparams['dataset'])
X = np.array(df_features[hparams['features']], dtype=np.float32)

encoder = LabelEncoder()
y = encoder.fit_transform(df_features['label'])

In [65]:
#K-fold Cross Validation model evaluation
k_folds = 8
kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)
indices = np.arange(len(y))
results_train = []
results_test = []

def train(loader_train, model, optimizer, epoch):
    model.train()
    running_correct = 0.0
    total = 0
    for batch_ndx, sample in enumerate(loader_train): 
        features, labels = sample[0].to(device), sample[1].to(device) 

        # forward pass and loss calculation
        outputs = model(features)
        loss = criterion(outputs, labels)  
        
        # backward pass    
        loss.backward()
        
        # update weights
        optimizer.step()
        optimizer.zero_grad()

        # calculate metrics
        predictions = torch.argmax(outputs.data, 1)
        running_correct += (predictions == labels).sum().item()
        total += labels.shape[0]

    return running_correct / total

def evaluate(loader_test, model, epoch):
    model.eval()
    running_correct = 0.0
    total = 0
    with torch.no_grad():
        for batch_ndx, sample in enumerate(loader_test):
            features, labels = sample[0].to(device), sample[1].to(device) 

            # forward pass and loss calculation
            outputs = model(features)
            loss = criterion(outputs, labels)  

            # calculate metrics
            predictions = torch.argmax(outputs.data, 1)
            running_correct += (predictions == labels).sum().item()
            total += labels.shape[0]

    return running_correct / total

print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
print('--------------------------------------------')
print('|         | Train Accuracy | Test Accuracy |')
print('--------------------------------------------')

for fold, (train_ids, test_ids) in enumerate(kfold.split(indices)):
    X_train = X[train_ids]
    y_train = y[train_ids]
    X_test = X[test_ids]
    y_test = y[test_ids]
    
    # scale data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # create pytorch dataloader
    torch.manual_seed(42)
    train_dataset = torch.utils.data.TensorDataset(torch.Tensor(X_train), torch.Tensor(y_train).long())
    test_dataset = torch.utils.data.TensorDataset(torch.Tensor(X_test), torch.Tensor(y_test).long())
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=hparams['batch_size'], shuffle=True)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=hparams['batch_size'], shuffle=False)
    
    # create model
    model = CoughNet(len(hparams['features'])).to(device)

    # Construct loss and optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=hparams['lr'])
    criterion = torch.nn.CrossEntropyLoss()

    # training loop
    for epoch in range(hparams['epochs']):
        train_accuracy = train(train_loader, model, optimizer, epoch)
        eval_accuracy = evaluate(test_loader, model, epoch)
    results_train.append(train_accuracy) 
    results_test.append(eval_accuracy) 
    print(f'| Fold {fold}  |       {train_accuracy*100:.2f} % |       {eval_accuracy*100:.2f} % |')

print('--------------------------------------------')
print(f'| Average |       {np.mean(results_train)*100:.2f} % |       {np.mean(results_test)*100:.2f} % |')

K-FOLD CROSS VALIDATION RESULTS FOR 8 FOLDS
--------------------------------------------
|         | Train Accuracy | Test Accuracy |
--------------------------------------------
| Fold 0  |       100.00 % |       94.12 % |
| Fold 1  |       100.00 % |       88.24 % |
| Fold 2  |       100.00 % |       94.12 % |
| Fold 3  |       100.00 % |       76.47 % |
| Fold 4  |       100.00 % |       88.24 % |
| Fold 5  |       100.00 % |       94.12 % |
| Fold 6  |       100.00 % |       100.00 % |
| Fold 7  |       96.61 % |       87.50 % |
--------------------------------------------
| Average |       99.58 % |       90.35 % |


In [66]:
#Training and Evaluation
#Prepare Data
df_features = pd.read_csv(hparams['dataset'])

X = np.array(df_features[hparams['features']], dtype=np.float32)

encoder = LabelEncoder()
y = encoder.fit_transform(df_features['label'])
print('classes:', encoder.classes_)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print('X_train.shape:', X_train.shape)
print('y_train.shape:', y_train.shape)

classes: ['covid' 'not_covid']
X_train.shape: (107, 26)
y_train.shape: (107,)


In [71]:
def train_eval_classifier(clf):
    clf.fit(X_train, y_train)

    predictions = clf.predict(X_train)
    accuracy_train = np.sum(predictions == y_train) / len(y_train)
    print("Train Accuracy:", accuracy_train)

    predictions = clf.predict(X_test)
    accuracy_test = np.sum(predictions == y_test) / len(y_test)
    print("Test Accuracy:", accuracy_test)

    plot_confusion_matrix(y_test, predictions, encoder.classes_)

def k_fold_train_eval_classifier(clf):    
    k_folds = 4
    kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)
    indices = np.arange(len(y))

    results_train = []
    results_test = []

    print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
    print('--------------------------------------------')
    print('|         | Train Accuracy | Test Accuracy |')
    print('--------------------------------------------')

    for fold, (train_ids, test_ids) in enumerate(kfold.split(indices)):
        X_train = X[train_ids]
        y_train = y[train_ids]
        X_test = X[test_ids]
        y_test = y[test_ids]

        # train classifier
        clf.fit(X_train, y_train)

        # evaluate classifier on train dataset
        predictions = clf.predict(X_train)
        train_accuracy = np.sum(predictions == y_train) / len(y_train)
        results_train.append(train_accuracy) 

        # evaluate classifier on test dataset
        predictions = clf.predict(X_test)
        eval_accuracy = np.sum(predictions == y_test) / len(y_test)        
        results_test.append(eval_accuracy) 

        print(f'| Fold {fold}  |       {train_accuracy*100:.2f} % |       {eval_accuracy*100:.2f} % |')

    print('--------------------------------------------')
    print(f'| Average |       {np.mean(results_train)*100:.2f} % |       {np.mean(results_test)*100:.2f} % |')
    
    plot_confusion_matrix(y_test, predictions, encoder.classes_)

In [72]:
#Naive Bayes
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()
k_fold_train_eval_classifier(clf)

K-FOLD CROSS VALIDATION RESULTS FOR 4 FOLDS
--------------------------------------------
|         | Train Accuracy | Test Accuracy |
--------------------------------------------
| Fold 0  |       80.00 % |       70.59 % |
| Fold 1  |       80.00 % |       61.76 % |
| Fold 2  |       74.26 % |       87.88 % |
| Fold 3  |       73.27 % |       81.82 % |
--------------------------------------------
| Average |       76.88 % |       75.51 % |


In [73]:
#Support Verctor Machine
from sklearn import svm

clf = svm.NuSVC(kernel='poly')
k_fold_train_eval_classifier(clf)

K-FOLD CROSS VALIDATION RESULTS FOR 4 FOLDS
--------------------------------------------
|         | Train Accuracy | Test Accuracy |
--------------------------------------------
| Fold 0  |       86.00 % |       73.53 % |
| Fold 1  |       79.00 % |       70.59 % |
| Fold 2  |       80.20 % |       63.64 % |
| Fold 3  |       78.22 % |       84.85 % |
--------------------------------------------
| Average |       80.85 % |       73.15 % |


In [74]:
#RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=10, n_estimators=100, random_state=42)
k_fold_train_eval_classifier(clf)

K-FOLD CROSS VALIDATION RESULTS FOR 4 FOLDS
--------------------------------------------
|         | Train Accuracy | Test Accuracy |
--------------------------------------------
| Fold 0  |       100.00 % |       82.35 % |
| Fold 1  |       100.00 % |       76.47 % |
| Fold 2  |       100.00 % |       90.91 % |
| Fold 3  |       100.00 % |       87.88 % |
--------------------------------------------
| Average |       100.00 % |       84.40 % |


In [26]:
#Artificial NeuralNetwork with RELU Activation
#Loading CSV file
train_csv = pd.read_csv("data/cough_trial_extended.csv")
train_csv

,file_properties,class
0,0v8MGxNetjg_ 10.000_ 20.000.wav,not_covid
1,1j1duoxdxBg_ 70.000_ 80.000.wav,not_covid
2,1MSYO4wgiag_ 120.000_ 130.000.wav,not_covid
3,1PajbAKd8Kg_ 0.000_ 10.000.wav,not_covid
4,cov1.wav,covid
...,...,...
165,-bZrDCS8KAg_ 70.000_ 80.000.wav,not_covid
166,-ej81N6Aqo4_ 0.000_ 8.000.wav,not_covid
167,-gvLnl1smfs_ 90.000_ 100.000.wav,not_covid
168,-hu5q-Nn4BM_ 70.000_ 80.000.wav,not_covid


In [27]:
train_csv['class'].unique()

array(['not_covid', 'covid'], dtype=object)

In [28]:
cmap = plt.get_cmap('inferno')
tot_rows = train_csv.shape[0]
for i in range(tot_rows):
    source = train_csv['file_properties'][i]
    filename = 'data/trial_covid/'+source
    y,sr = librosa.load(filename, mono=True, duration=5)
    plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
    plt.axis('off');
    #plt.savefig(f'./{source[:-3].replace(".", "")}.png')
    #plt.savefig(f'data/plot/{source[:-3].replace(".", "")}.png')
    plt.show()
    #plt.clf()

In [29]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [30]:
header

['filename',
 'chroma_stft',
 'rmse',
 'spectral_centroid',
 'spectral_bandwidth',
 'rolloff',
 'zero_crossing_rate',
 'mfcc1',
 'mfcc2',
 'mfcc3',
 'mfcc4',
 'mfcc5',
 'mfcc6',
 'mfcc7',
 'mfcc8',
 'mfcc9',
 'mfcc10',
 'mfcc11',
 'mfcc12',
 'mfcc13',
 'mfcc14',
 'mfcc15',
 'mfcc16',
 'mfcc17',
 'mfcc18',
 'mfcc19',
 'mfcc20',
 'label']

In [31]:
file = open('data/data_new_extended.csv', 'w')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
for i in range(tot_rows):
        source = train_csv['file_properties'][i]
        file_name = 'data/trial_covid/'+source
        y,sr = librosa.load(file_name, mono=True, duration=5)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        rmse = librosa.feature.rms(y=y)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{source[:-3].replace(".", "")} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        
        file = open('data/data_new_extended.csv', 'a')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

In [32]:
data1 = pd.read_csv('data/data_new_extended.csv')
data1

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0v8MGxNetjg_,10000_,20000.000000,0.519951,0.045853,1612.895795,1411.838677,2907.580566,0.107019,-376.876007,111.017372,...,-0.656475,-7.439712,-1.034580,-0.203083,-3.513495,-1.745705,-3.011878,-2.878482,-2.106427,-4.026825
1j1duoxdxBg_,70000_,80000.000000,0.535472,0.001771,2892.087076,2467.408141,5072.664388,0.148584,-519.158447,60.781284,...,-0.156307,-0.909973,7.216461,-1.719629,3.903021,3.653039,3.043882,2.439957,2.781968,2.195162
1MSYO4wgiag_,120000_,130000.000000,0.496666,0.033657,3429.061935,2788.634413,6886.288452,0.225315,-282.297913,48.581680,...,0.829615,-6.066336,-4.167640,1.017302,-0.523806,0.538693,-8.855953,-2.927977,-1.118562,-5.906228
1PajbAKd8Kg_,0000_,10000.000000,0.407549,0.013452,2710.811637,2664.287550,5778.474935,0.142076,-346.857300,75.765617,...,-2.838680,5.053118,-0.291308,0.987186,-2.447526,3.692367,2.312328,-2.059656,-4.772599,-0.503851
cov1,0.41269657015800476,0.059004,1555.648634,1418.599932,2870.737092,0.133998,-340.588013,104.156700,-32.228443,-13.615362,...,0.940006,-5.701087,-6.326630,-1.080040,-1.812609,-2.518986,-3.684266,-3.564146,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
-bZrDCS8KAg_,70000_,80000.000000,0.492974,0.005093,1600.647469,2300.999728,3660.644531,0.047815,-543.776917,119.100296,...,-2.675646,-1.250754,-2.634280,1.647435,0.647164,1.602689,-2.469729,0.704325,-5.352920,-1.281080
-ej81N6Aqo4_,0000_,8000.000000,0.400283,0.052132,2664.129566,2563.440387,5518.182373,0.121514,-290.840607,85.514404,...,-8.843078,-4.629812,-7.424622,-4.511141,-7.482200,-4.865530,-6.353733,-5.024187,-8.422812,-0.831208
-gvLnl1smfs_,90000_,100000.000000,0.704281,0.058739,3090.031219,2740.856272,6530.841064,0.179077,-75.595451,68.849228,...,-6.867559,0.677697,-7.535110,0.602187,-6.629556,0.659050,-4.125255,0.734950,-4.655417,-0.645009
-hu5q-Nn4BM_,70000_,80000.000000,0.424896,0.044159,3173.872023,2482.951387,5768.306478,0.221743,-264.064514,58.729767,...,-3.354259,-0.625627,0.677355,-3.651989,-6.051376,1.211774,-14.923816,-11.180058,-8.861263,-5.078876


In [33]:
import seaborn as sns
d = data1.drop(['filename','label'],axis=1)
h = sns.heatmap(d)

In [34]:
data1.shape

(170, 28)

In [35]:
# Dropping unneccesary columns
data1 = data1.drop(['filename'],axis=1)

In [36]:
genre_list = data1.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

In [37]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data1.iloc[:, :-1], dtype = float))

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [39]:
X_train

array([[ 0.28758994, -0.3478334 , -0.35099591, ...,  0.98243227,
        -1.40798036,  0.65523484],
       [-0.22354755, -0.34822437, -0.35097655, ..., -1.63453015,
        -1.67422902, -1.63924591],
       [ 0.67094305, -0.34780793, -0.35099899, ...,  0.52941914,
         0.52278956,  0.11991464],
       ...,
       [-0.59412223, -0.34792136, -0.35097261, ...,  0.34110886,
         0.54303798,  0.29167731],
       [-0.73468503, -0.34796061, -0.35102339, ..., -0.64290737,
         0.56478156,  0.02234052],
       [ 1.69321803, -0.3480218 , -0.35098855, ...,  1.80197436,
        -0.40673473,  3.97283816]])

In [40]:
import tensorflow as tf
from tensorflow import keras
from keras import models
from keras import layers
from keras.layers import Dropout
from keras.utils.vis_utils import plot_model


model = tf.keras.Sequential()
model.add(layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.3, input_shape=(60,)))

model.add(layers.Dense(64, activation='relu'))

#model.add(layers.Dense(128, activation='relu'))

#model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(10, activation='relu'))

model.add(layers.Dense(1, activation='sigmoid'))

# serialize model to JSON
#model_json = model.to_json()
#with open("model.json", "w") as json_file:
    #json_file.write(model_json)
# serialize weights to HDF5
#model.save_weights("model.h5")
print("Saved model to disk")

# plot model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

Saved model to disk
('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [41]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               3456      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                650       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 12,373
Trainable params: 12,373
Non-trainable params: 0
_________________________________________________________________


In [42]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(X_train, y_train, epochs=15)

Epoch 1/15
5/5 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0074
Epoch 2/15
5/5 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0074
Epoch 3/15
5/5 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0074
Epoch 4/15
5/5 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0074
Epoch 5/15
5/5 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0074
Epoch 6/15
5/5 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0074
Epoch 7/15
5/5 [==============================] - 0s 3ms/step - loss: nan - accuracy: 0.0074
Epoch 8/15
5/5 [==============================] - 0s 3ms/step - loss: nan - accuracy: 0.0074
Epoch 9/15
5/5 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0074
Epoch 10/15
5/5 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0074
Epoch 11/15
5/5 [==============================] - 0s 2ms/step - loss

In [43]:
test_loss, test_acc = model.evaluate(X_test,y_test)

2/2 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.0000e+00
